# gnina docking

## Setup

In [ ]:
#| default_exp gnina_dock

In [ ]:
#| export
# basics
import re,subprocess, py3Dmol
from tqdm import tqdm
from pathlib import Path
import pandas as pd,numpy as np

# rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

## Setup gnina

Below are the commands to setup

In [ ]:
# !sudo apt install -yq openbabel
# !wget https://github.com/gnina/gnina/releases/download/v1.0.3/gnina #check the latest version please
# !chmod +x gnina
# !./gnina

Alternatively, use python script to setup:

In [ ]:
#| export
def setup_gnina_local(version='v1.3'):
    "Download and install gnina in the current directory"
    # Check CUDA availability
    # try:
    #     subprocess.run(["nvidia-smi"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # except subprocess.CalledProcessError:
    #     raise EnvironmentError("CUDA not detected. Please make sure a CUDA-capable GPU is available and drivers are installed.")
    # except FileNotFoundError:
    #     raise EnvironmentError("nvidia-smi not found. Make sure NVIDIA drivers and CUDA are installed.")

    subprocess.run(["sudo", "apt-get", "update", "-yq"], check=True,stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["sudo", "apt-get", "install", "-yq", "openbabel"], check=True)

    gnina_url = f"https://github.com/gnina/gnina/releases/download/{version}/gnina"
    print(f'Downloading {version} gnina')
    subprocess.run(["wget",gnina_url], check=True,stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["chmod", "+x", 'gnina'], check=True)
    
    print('Finish setup!')

In [ ]:
# setup_gnina_local()

Reading package lists...
Building dependency tree...
Reading state information...
openbabel is already the newest version (3.0.0+dfsg-3ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Finish setup!


In [ ]:
def delete_gnina(gnina_path="gnina"):
    "Remove gnina after finish"
    path=Path(gnina_path)
    if path.exists() and path.is_file():
        path.unlink()
        print('Deleted!')
    else: print('Not exist!')

In [ ]:
# delete_gnina()

Deleted!


If local is not successful, use docker

In [ ]:
#| export
def setup_gnina_docker():
    "Pull gnina docker image"
    print("Pulling GNINA Docker image: gnina/gnina")
    subprocess.run(["docker", "pull", "gnina/gnina"], check=True,stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("GNINA Docker image is ready.")

In [ ]:
# setup_gnina_docker()

Pulling GNINA Docker image: gnina/gnina
GNINA Docker image is ready.


## Get protein and ligand

In [ ]:
# #nature medicine paper, mrtx
# !wget http://files.rcsb.org/download/7T47.pdb

# # get ligand
# !grep 6IC 7T47.pdb > lig.pdb

# # get protein
# # !grep ATOM 7T47.pdb > rec.pdb #not as good as the one below

# !grep -v 6IC 7T47.pdb > rec.pdb

## Conformer generation

In [ ]:
#| export
def rdkit_conformer(SMILES, # SMILES string
                    output, # file ".sdf" to be saved
                    method='ETKDG', # Optimization method, can be 'UFF', 'MMFF' or 'ETKDGv3'
                    visualize=True, #whether or not to visualize the compound
                    seed = 3,# randomness of the 3d
                    ):
    
    np.random.seed(seed) 
    mol = Chem.MolFromSmiles(SMILES)
    
    # Generate a 3D conformation of the molecule
    AllChem.EmbedMolecule(mol)
    

    # Optimize the 3D conformation using the specified force field method
    if method == 'UFF':
        AllChem.UFFOptimizeMolecule(mol)
    elif method == 'MMFF':
        AllChem.MMFFOptimizeMolecule(mol)
    elif method == 'ETKDG':
        AllChem.EmbedMultipleConfs(mol, numConfs=1, useExpTorsionAnglePrefs=True, 
                                   useBasicKnowledge=True, enforceChirality=True,randomSeed=seed)
        AllChem.ETKDGv3()
        AllChem.UFFOptimizeMolecule(mol)
        
#     elif method == 'ETKDG_MMFF':
#         AllChem.EmbedMultipleConfs(mol, numConfs=1, useExpTorsionAnglePrefs=True, 
#                                    useBasicKnowledge=True, enforceChirality=True)
#         AllChem.ETKDGv3()
#         AllChem.MMFFOptimizeMolecule(mol)
    else:
        raise ValueError('Invalid method specified')
        

    # Remove hydrogens from the molecule
    # mol = Chem.RemoveHs(mol)
    
    Path(output).parent.mkdir(parents=True,exist_ok=True)

    w = Chem.SDWriter(output)
    w.write(mol)
    w.close()
    
    if visualize:
        v = py3Dmol.view()
        v.addModel(Chem.MolToMolBlock(mol))
        v.setStyle({'stick':{}})
        v.zoomTo()
        v.show()

## Gnina affinity

In [ ]:
#| export
def extract_gnina_dock(gnina_output):
    "Extract values from gnina output"
    mode1_line = re.search(r'\b1\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\b', gnina_output)
    
    if mode1_line:
        affinity = float(mode1_line.group(1))
        cnn_pose_score = float(mode1_line.group(2))
        cnn_affinity = float(mode1_line.group(3))
        
        return affinity, cnn_pose_score, cnn_affinity
    
    return None

In [ ]:
#| export
def gnina_dock(receptor, # receptor file
              ligand, # ligand file
              autobox_ligand, # ligand file isolated from the complex
              output = 'docked.sdf', # output file (sdf or sdf.gz) to be saved
              seed=0, # set seeds
              exhaustiveness=None, # number of MC chains, default is 8 if None, the higher the better (16,32); for whole protein, use 64
              ):
    
    command = ['./gnina', 
               '-r', receptor, 
               '-l', ligand, 
               '--autobox_ligand', autobox_ligand,
               '-o', output,
               '--seed', str(seed)]

    if exhaustiveness is not None:
        command.extend(['--exhaustiveness', str(exhaustiveness)])

    output_txt = subprocess.run(command, capture_output=True, text=True).stdout
    
    print(f'save the docked file as {output}')
    
    values = extract_gnina_dock(output_txt)
    
    print(f'affinity, cnn_pose_score, and cnn_affinity are: {values}')

    return values

In [ ]:
# df = Data.get_g12d_dedup()[:10]

In [ ]:
#| export
def gnina_dock(df, 
                 ID_col = 'ID',
                 smi_col = 'SMILES',
              output_dir = 'gnina_docked'
                 ):
    affinity_values = []
    cnn_pose_score_values = []
    cnn_affinity_values = []

    
    Path(output_dir).mkdir(parents=True,exist_ok=True)

    for i, r in tqdm(df.iterrows(),total=len(df),desc='Docking'):
        rdkit_conformer(SMILES=r[smi_col], output = f'ligand/{r[ID_col]}.sdf', visualize=False)
        affinity, cnn_pose_score, cnn_affinity = gnina_dock('rec.pdb',f'ligand/{r[ID_col]}.sdf', 'lig.pdb',f'docked/docked_{r[ID_col]}.sdf')

        affinity_values.append(affinity)
        cnn_pose_score_values.append(cnn_pose_score)
        cnn_affinity_values.append(cnn_affinity)

    df = df.copy()
    df['Affinity'] = affinity_values
    df['CNN_Pose_Score'] = cnn_pose_score_values
    df['CNN_Affinity'] = cnn_affinity_values
    
    return df

In [ ]:
# out = gnina_dock_df(df)

TODO: make it parallel

## End

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()